In [1]:
import pandas as pd
import numpy as np
import os
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from pprint import pprint

/home/giotto/.local/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


**Dataset configuration**

In [2]:
dataset = 'robotics-final'

dataset_path = '../datasets/' + dataset + '/'
output_path = '../datasets/' + dataset + '-2s-processed/'
features_output_path = '../datasets/' + dataset + '-2s-features/'

In [3]:
devices_to_use = [
    '128.237.246.127',
    '128.237.248.186',
    '128.237.247.134',
    '128.237.234.0',
    '128.237.237.122',
    '128.237.239.234',
    '128.237.254.195', # this Mite is only in the Synergy kitchen dataset
    'DialogIoT 591844595',
    'DialogIoT 591844599',
    'DialogIoT 591844765',
    'Matrix b827eb96f31a',
    'Matrix b827ebe6e0f8',
    'Matrix b827eb41f96f',
    'TI SensorTag 604',
    'TI SensorTag 690',
    'TI SensorTag 85',
    'xdk_1',
    'xdk_2',
    'xdk_3',
    'TI SensorTag 33',
    'TI SensorTag 535',
    'TI SensorTag 709'
]

columns_to_rename = {
    'ACCEL_sst_0_avg': 'accel_x',
    'ACCEL_sst_1_avg': 'accel_y',
    'ACCEL_sst_2_avg': 'accel_z',
    'MAGNETOMETER_sst_0_avg': 'mag_x',
    'MAGNETOMETER_sst_1_avg': 'mag_y',
    'MAGNETOMETER_sst_2_avg': 'mag_z',
    'HUMIDITY_sst_0_avg': 'humidity',
    'ILLUMINATION_sst_0_avg': 'light',
    'BAROMETER_sst_0_avg': 'pressure',
    'TEMPERATURE_sst_0_avg': 'temperature',
    'MICROPHONE_sst_0_avg': 'microphone',
    'MICROPHONE_sst_0_min': 'microphone_min',
    'MICROPHONE_sst_0_max': 'microphone_max',
    'MICROPHONE_sst_0_sum': 'microphone_sum',
    'MICROPHONE_sst_0_variance': 'microphone_variance',
    'MICROPHONE_sst_0_range': 'microphone_range',
    'MICROPHONE_sst_0_centroid': 'microphone_centroid',
    'microphone_avg': 'microphone',
    'magnetometer_x': 'mag_x',
    'magnetometer_y': 'mag_y',
    'magnetometer_z': 'mag_z'
}

columns_to_keep = '|'.join([
    'accel_',
    'gyro_',
    'mag_',
    'humidity',
    'light',
    'pressure',
    'temperature',
    'microphone',
#     'ACCEL_fft_',
#     'MICROPHONE_fft_',
])

**Merge the individual CSVs into one DataFrame per device for the chosen devices**

In [4]:
data_frames = {}

for root, dirs, files in os.walk(dataset_path):
    for file in files:
        device_id = None

        for device in devices_to_use:
            if file.startswith(device):
                device_id = device

        if device_id is not None:
            df = pd.DataFrame()
            if device_id in data_frames:
                df = data_frames[device_id]
            
            df_new = pd.DataFrame.from_csv(dataset_path + file)
            df_new.rename(index=str, columns=columns_to_rename, inplace=True)

            df_new = df_new.filter(regex=(columns_to_keep))

            if len(df_new.columns) > 0:
                df = df.join(df_new, how='outer')

                data_frames[device_id] = df

**Remove duplicate indices**

In [5]:
for device_id in data_frames:
    df = data_frames[device_id]
    df = df[~df.index.duplicated(keep='first')]
    data_frames[device_id] = df

**Convert index to datetime**

In [6]:
for device_id in data_frames:
    df = data_frames[device_id]
    df.index = pd.to_datetime(df.index)
    data_frames[device_id] = df

**Fill missing values**

In [7]:
for device_id in data_frames:
    df = data_frames[device_id]
    df = df.ffill().bfill()
    data_frames[device_id] = df

**Create 5 sec activity windows**

In [8]:
window_size_secs = 2 # seconds
smallest_size_secs = 1 # seconds

activities = pd.DataFrame.from_csv(dataset_path + 'activities.csv')
labels = pd.DataFrame.from_csv(dataset_path + 'activity_labels.csv')
labeled_activities = activities.loc[activities.id != -1]

activity_windows = []

for id in labeled_activities.id.unique():
    start = labeled_activities.loc[labeled_activities.id == id].index.min()
    end = labeled_activities.loc[labeled_activities.id == id].index.max()

    since = start
    until = start + pd.DateOffset(seconds=window_size_secs)

    while since < end:
        label = labels.loc[id]['label']

        length = (until - since).seconds
        if length >= smallest_size_secs:
            activity_windows.append({
                'since': since,
                'until': until,
                'label': label,
                'length': length,
                'id': id
            })

        since = until
        until = until + pd.DateOffset(seconds=window_size_secs)
        until = min(until, end)

**Create per-activity dataframes which are not missing more than 1 second of data**

In [9]:
activity_data_frames = {}
device_labels = {}

removed_activities = {}
all_activities = {}
activity_id = 0

for device_id in data_frames:
    df = data_frames[device_id]
    data_frames[device_id] = None
    df['id'] = -1

    device_activities = []

    for i, window in enumerate(activity_windows):

        activity_df = df.loc[df.index >= window['since']]
        activity_df = activity_df.loc[activity_df.index <= window['until']].copy()

        seconds = activity_df.index.round('s').unique()

        key = device_id + ', activity ' + str(window['label'])
        if not key in all_activities:
            all_activities[key] = 0
            removed_activities[key] = 0

        if len(seconds) < window['length']:
            removed_activities[key] += 1
        else:
            activity_df['id'] = activity_id
            activity_data_frames[activity_id] = activity_df
            device_activities.append(pd.DataFrame({
                'id': [activity_id],
                'label': [window['label']],
                'activity_id': [window['id']],
                'since': [window['since']],
                'until': [window['until']],
                'window_id': [i]
            }))
            activity_id += 1

        all_activities[key] += 1

    df_labels = pd.concat(device_activities)
    df_labels = df_labels.set_index('id')
    device_labels[device_id] = df_labels

# print stats    
for key in all_activities:
    if removed_activities[key] > 0:
        print(str(removed_activities[key]) + ' out of ' + str(all_activities[key]) + ' removed for ' + key)

1 out of 184 removed for TI SensorTag 85, activity 1
2 out of 186 removed for DialogIoT 591844595, activity 4
1 out of 182 removed for TI SensorTag 604, activity 0
1 out of 107 removed for TI SensorTag 604, activity 16
69 out of 187 removed for DialogIoT 591844599, activity 8
1 out of 196 removed for TI SensorTag 85, activity 5
3 out of 184 removed for 128.237.246.127, activity 1
2 out of 184 removed for TI SensorTag 604, activity 1


**Resample to 10Hz and fill missing values**

In [10]:
for activity_id in activity_data_frames:
    df = activity_data_frames[activity_id]
    df = df.resample('100L')
    df = df.ffill().bfill()
    activity_data_frames[activity_id] = df

**Update device data frames**

In [11]:
for device_id in device_labels:
    labels = device_labels[device_id]
    df_parts = [activity_data_frames[id] for id in labels.index]
    df = pd.concat(df_parts)
    data_frames[device_id] = df

**Replace timestamp indices with integer ones**

In [12]:
for device_id in data_frames:
    df = data_frames[device_id]
    df.insert(0, 'i_index', pd.Series(range(len(df.index)), index=df.index))
    df = df.set_index(['i_index'])
    data_frames[device_id] = df

**Save as Pickle files to disk**

In [ ]:
for device_id in data_frames:
    df = data_frames[device_id]
    df.to_pickle(output_path + device_id + '.p')

    df_labels = device_labels[device_id]
    df_labels.to_pickle(output_path + device_id + '_labels.p')

**Reload extracted datasets from disk**

In [14]:
data_frames = {}
device_labels = {}

for root, dirs, files in os.walk(output_path):
    for file in files:
        device_id = None

        for device in devices_to_use:
            if file.startswith(device):
                device_id = device

        if device_id is not None:
            df = pd.read_pickle(output_path + file)
            if file.endswith('_labels.p'):
                device_labels[device_id] = df
            else:
                data_frames[device_id] = df

**Extract features**

In [ ]:
data_frame_features = {}

def extract_device_features(df):
    extracted_features = extract_features(impute(df), column_id="id")
    impute(extracted_features)
    return extracted_features

for device_id in data_frames:
    df = data_frames[device_id]

    extracted = extract_device_features(df)
    data_frame_features[device_id] = extracted

Feature Extraction: 100%|██████████| 18/18 [04:34<00:00,  6.81s/it]
 'microphone_centroid__friedrich_coefficients__m_3__r_30__coeff_3'
 'microphone_centroid__friedrich_coefficients__m_3__r_30__coeff_2'
 'microphone_centroid__friedrich_coefficients__m_3__r_30__coeff_1'
 'microphone_centroid__friedrich_coefficients__m_3__r_30__coeff_0'
 'temperature__max_langevin_fixed_point__m_3__r_30'
 'temperature__friedrich_coefficients__m_3__r_30__coeff_3'
 'temperature__friedrich_coefficients__m_3__r_30__coeff_2'
 'temperature__friedrich_coefficients__m_3__r_30__coeff_1'
 'temperature__friedrich_coefficients__m_3__r_30__coeff_0'
 'pressure__max_langevin_fixed_point__m_3__r_30'
 'pressure__friedrich_coefficients__m_3__r_30__coeff_3'
 'pressure__friedrich_coefficients__m_3__r_30__coeff_2'
 'pressure__friedrich_coefficients__m_3__r_30__coeff_1'
 'pressure__friedrich_coefficients__m_3__r_30__coeff_0'
 'microphone_sum__max_langevin_fixed_point__m_3__r_30'
 'microphone_sum__friedrich_coefficients__m_3__r

Feature Extraction: 100%|██████████| 14/14 [04:24<00:00, 21.23s/it]
 'humidity__friedrich_coefficients__m_3__r_30__coeff_3'
 'humidity__friedrich_coefficients__m_3__r_30__coeff_2'
 'humidity__friedrich_coefficients__m_3__r_30__coeff_1'
 'humidity__friedrich_coefficients__m_3__r_30__coeff_0'
 'mag_z__max_langevin_fixed_point__m_3__r_30'
 'mag_z__friedrich_coefficients__m_3__r_30__coeff_3'
 'mag_z__friedrich_coefficients__m_3__r_30__coeff_2'
 'mag_z__friedrich_coefficients__m_3__r_30__coeff_1'
 'mag_z__friedrich_coefficients__m_3__r_30__coeff_0'
 'mag_y__max_langevin_fixed_point__m_3__r_30'
 'mag_y__friedrich_coefficients__m_3__r_30__coeff_3'
 'mag_y__friedrich_coefficients__m_3__r_30__coeff_2'
 'mag_y__friedrich_coefficients__m_3__r_30__coeff_1'
 'mag_y__friedrich_coefficients__m_3__r_30__coeff_0'
 'temperature__max_langevin_fixed_point__m_3__r_30'
 'temperature__friedrich_coefficients__m_3__r_30__coeff_3'
 'temperature__friedrich_coefficients__m_3__r_30__coeff_2'
 'temperature__friedri

Feature Extraction: 100%|██████████| 10/10 [02:55<00:00, 14.44s/it]
 'mag_y__autocorrelation__lag_6' 'mag_y__augmented_dickey_fuller'
 'mag_y__sample_entropy' 'mag_y__max_langevin_fixed_point__m_3__r_30'
 'mag_y__autocorrelation__lag_7' 'mag_y__autocorrelation__lag_8'
 'mag_y__autocorrelation__lag_1' 'mag_y__autocorrelation__lag_3'
 'mag_y__autocorrelation__lag_4' 'mag_y__autocorrelation__lag_9'
 'mag_y__autocorrelation__lag_0'
 'mag_y__friedrich_coefficients__m_3__r_30__coeff_3'
 'mag_y__friedrich_coefficients__m_3__r_30__coeff_2'
 'mag_y__friedrich_coefficients__m_3__r_30__coeff_1'
 'mag_y__friedrich_coefficients__m_3__r_30__coeff_0'
 'accel_y__max_langevin_fixed_point__m_3__r_30'
 'accel_y__friedrich_coefficients__m_3__r_30__coeff_3'
 'accel_y__friedrich_coefficients__m_3__r_30__coeff_2'
 'accel_y__friedrich_coefficients__m_3__r_30__coeff_1'
 'accel_y__friedrich_coefficients__m_3__r_30__coeff_0'
 'gyro_y__max_langevin_fixed_point__m_3__r_30'
 'gyro_y__friedrich_coefficients__m_3__r_

Feature Extraction: 100%|██████████| 10/10 [02:57<00:00, 13.66s/it]
 'accel_z__friedrich_coefficients__m_3__r_30__coeff_3'
 'accel_z__friedrich_coefficients__m_3__r_30__coeff_2'
 'accel_z__friedrich_coefficients__m_3__r_30__coeff_1'
 'accel_z__friedrich_coefficients__m_3__r_30__coeff_0'
 'gyro_y__max_langevin_fixed_point__m_3__r_30'
 'gyro_y__friedrich_coefficients__m_3__r_30__coeff_3'
 'gyro_y__friedrich_coefficients__m_3__r_30__coeff_2'
 'gyro_y__friedrich_coefficients__m_3__r_30__coeff_1'
 'gyro_y__friedrich_coefficients__m_3__r_30__coeff_0'
 'accel_y__max_langevin_fixed_point__m_3__r_30'
 'accel_y__friedrich_coefficients__m_3__r_30__coeff_3'
 'accel_y__friedrich_coefficients__m_3__r_30__coeff_2'
 'accel_y__friedrich_coefficients__m_3__r_30__coeff_1'
 'accel_y__friedrich_coefficients__m_3__r_30__coeff_0'
 'mag_y__max_langevin_fixed_point__m_3__r_30'
 'mag_y__friedrich_coefficients__m_3__r_30__coeff_3'
 'mag_y__friedrich_coefficients__m_3__r_30__coeff_2'
 'mag_y__friedrich_coefficien

**Sort feature columns**

In [ ]:
for device_id in data_frame_features:
    df = data_frame_features[device_id]
    df = df.reindex_axis(sorted(df.columns), axis=1)
    data_frame_features[device_id] = df

**Store extracted features on disk**

In [ ]:
for device_id in data_frame_features:
    extracted = data_frame_features[device_id]
    extracted.to_pickle(features_output_path + device_id + '.p')

    df_labels = device_labels[device_id]
    df_labels.to_pickle(features_output_path + device_id + '_labels.p')

**Do feature selection**

In [ ]:
data_frame_selected = {}

for device_id in device_labels:
    df = data_frame_features[device_id]
    df_labels = device_labels[device_id]
    selected = select_features(df, df_labels['label'])
    data_frame_selected[device_id] = selected

Feature Selection:   0%|          | 0/3996 [00:00<?, ?it/s]WARNING:tsfresh.feature_selection.feature_selector:[test_feature_significance] Feature mag_z__autocorrelation__lag_7 is constant


Feature Selection:   0%|          | 1/3996 [00:00<14:18,  4.65it/s]WARNING:tsfresh.feature_selection.feature_selector:[test_feature_significance] Feature microphone_sum__mean_abs_change_quantiles__qh_0.6__ql_0.8 is constant


Feature Selection:   8%|▊         | 336/3996 [00:00<09:11,  6.64it/s]WARNING:tsfresh.feature_selection.feature_selector:[test_feature_significance] Feature microphone_max__autocorrelation__lag_9 is constant


Feature Selection:  11%|█         | 436/3996 [00:00<06:16,  9.45it/s]WARNING:tsfresh.feature_selection.feature_selector:[test_feature_significance] Feature gyro_x__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_12__w_5 is constant


Feature Selection:  34%|███▎      | 1341/3996 [00:00<03:16, 13.49it/s]WARNING:tsfresh.feature_selection.feature_selector:[test_feature_significance] Feature pressure__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_13__w_10 is constant


...

Feature Selection:   0%|          | 2/3108 [00:00<09:46,  5.29it/s]WARNING:tsfresh.feature_selection.feature_selector:[test_feature_significance] Feature gyro_z__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_10__w_10 is constant


Feature Selection:  33%|███▎      | 1041/3108 [00:00<04:33,  7.56it/s]WARNING:tsfresh.feature_selection.feature_selector:[test_feature_significance] Feature gyro_y__symmetry_looking__r_0.9500000000000001 is constant


Feature Selection:  53%|█████▎    | 1657/3108 [00:00<01:34, 15.37it/s]WARNING:tsfresh.feature_selection.feature_selector:[test_feature_significance] Feature accel_z__ar_coefficient__k_10__coeff_1 is constant


Feature Selection:  79%|███████▊  | 2445/3108 [00:00<00:30, 21.93it/s]WARNING:tsfresh.feature_selection.feature_selector:[test_feature_significance] Feature accel_y__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_14__w_2 is constant


Feature Selection: 100%|██████████| 3108/3108 [00:00<00:00, 3141.23it/s]WARNING:tsfresh.feature_selection.feature_selector:[test_feature_significance] Feature pressure__ar_coefficient__k_10__coeff_3 is constant


**Save to disk**

In [ ]:
for device_id in data_frame_selected:
    selected = data_frame_selected[device_id]
    selected.to_pickle(features_output_path + device_id + '_selected.p')

In [ ]:
1

1

In [15]:
1

1